In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [171]:
df = pd.read_csv("../data/train_data.csv")
df.isnull().sum()/df.shape[0]*100

age                         0.0
gender                      0.0
income_bracket              0.0
loyalty_program             0.0
membership_years            0.0
                           ... 
email_subscriptions         0.0
app_usage                   0.0
website_visits              0.0
social_media_engagement     0.0
days_since_last_purchase    0.0
Length: 77, dtype: float64

In [172]:
df.head()

,age,gender,income_bracket,loyalty_program,membership_years,churned,marital_status,number_of_children,education_level,occupation,...,distance_to_store,holiday_season,season,weekend,customer_support_calls,email_subscriptions,app_usage,website_visits,social_media_engagement,days_since_last_purchase
0,41,Female,Medium,No,3,No,Married,0,Bachelor's,Employed,...,72.83,No,Summer,No,18,No,Medium,2,Medium,6
1,40,Female,High,Yes,4,Yes,Single,4,PhD,Employed,...,44.60,Yes,Fall,No,16,Yes,Low,17,Medium,137
2,48,Male,High,No,4,Yes,Married,3,Bachelor's,Unemployed,...,46.15,Yes,Winter,Yes,11,Yes,High,68,High,275
3,19,Male,Low,No,9,Yes,Divorced,3,Bachelor's,Retired,...,0.62,No,Fall,No,15,Yes,Medium,61,Low,182
4,53,Female,Low,Yes,8,No,Divorced,2,Master's,Employed,...,38.09,Yes,Fall,Yes,0,Yes,High,19,High,330


In [173]:
# Get the number of unique values for each column
nunique_counts = df.nunique()

# Filter columns where nunique is less than 20
cols_less_than = nunique_counts[nunique_counts < 10]

cols_less_than.count()

np.int64(34)

In [174]:
date_cols = []

# collect date columns
for i in df.columns:
    if 'date' in i.lower():
        date_cols.append(i)

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
    df[col] = df[col].astype('int64') / 1e9  # convert to float seconds


In [175]:
to_drop = ['transaction_id']
df.drop(columns=date_cols, inplace=True)

In [176]:
categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
# Drop columns with all missing or blank values before encoding
df = df.dropna(axis=1, how='all')
df = df.loc[:, (df != '').any(axis=0)]
# One-hot encode if any categorical columns remain
if categorical_cols:
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=False, dtype=float)
# Drop all-zero columns after encoding
df = df.loc[:, (df != 0).any(axis=0)]
df.head()

,age,membership_years,number_of_children,transaction_id,product_id,quantity,unit_price,discount_applied,transaction_hour,week_of_year,...,weekend_No,weekend_Yes,email_subscriptions_No,email_subscriptions_Yes,app_usage_High,app_usage_Low,app_usage_Medium,social_media_engagement_High,social_media_engagement_Low,social_media_engagement_Medium
0,41,3,0,237855,997,7,960.62,0.24,0,48,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,40,4,4,500661,2869,5,219.52,0.46,12,19,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,48,4,3,432552,6172,8,759.95,0.11,22,1,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3,19,9,3,771343,6004,9,862.15,0.44,11,36,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,53,8,2,186219,3565,3,609.81,0.19,11,42,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [177]:
numeric_cols = df.select_dtypes(include=['number']).columns

#Outlier remover
def whisker(col):
    Q1,Q3 = np.percentile(col,[25,75])
    iqr = Q3 - Q1
    lw = Q1 - (1.5 * iqr)
    uw = Q3 + (1.5 * iqr)
    return lw, uw

In [178]:
for i in numeric_cols:
    lw,uw = whisker(df[i])
    df[i] = np.where(df[i]<lw, lw, df[i])
    df[i] = np.where(df[i]>uw, uw, df[i])
    # sns.boxplot(data=df, x=i)
    # plt.show()

In [179]:
# Data normalization
# numerical_cols.append("FlashStorage")
for col in numerical_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col] = (df[col] - min_val) / (max_val - min_val)

# for i in numeric_cols:
#     sns.scatterplot(data=df, x=i, y='Price')
#     plt.show())
# df.drop(columns=["FlashStorage"], inplace=True)


In [180]:
df = df.dropna(axis=1)

In [181]:
output_path = "/Users/avi/Desktop/university/sem_5/Machine_Learning/Avi_Dhall_A1/retail_task/data/cleaned_data.csv"
df.to_csv(output_path, index=False)